In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import MDAnalysis as mda
from MDAnalysis.analysis.waterdynamics import WaterOrientationalRelaxation as WOR
from scipy.optimize import curve_fit

In [ ]:
def read_topology_pdb(topo_f, atom_indicator="HETATM"):
    """
    Read a pdb file containing the topology of the system where an atom line begins by atom_indicator
    Return a dictionary idx_to_type such that: idx_to_type["index"] = type of the index-th atom
    """
    idx_to_type = {}
    
    f = open(topo_f, "r")
    for i,line in enumerate(f):
        if atom_indicator == line.split()[0]:
            idx = line.split()[1]  # 2nd column of pdb contains index
            atom_type = line.split()[2] # 3rd column of pdb contains atom type
            idx_to_type[idx]=atom_type
    
    f.close()
    return idx_to_type
    
def lammps_velocity_dump_to_xyz(lammps_f, idx_to_type, xyz_outname, start_coord_token="ITEM: ATOMS id vx vy vz"):
    """
    Read a custom lammps file containing the data indicated in start_coord_token
    Write xyz file xyz_outname with the velocity information and ordered atom indexes.
    Attention: the function requires the dictionary created by read_topology_pdb() on a topology file matching the system
    """
    # In lammps dumpfiles header info starts by the "ITEM: TIMESTEP" token and contains 9 lines
    header_token = "ITEM: TIMESTEP"
    header_lines = 9
    
    N_atoms = len(idx_to_type)
    
    # The token indicating the start of atom information starts always by ITEM: ATOMS
    # so we need to correct the index
    atom_id_index_in_line = start_coord_token.split().index("id") - 2
    vx_index_in_line = start_coord_token.split().index("vx") - 2
    vy_index_in_line = start_coord_token.split().index("vy") - 2
    vz_index_in_line = start_coord_token.split().index("vz") - 2
    
    # loop over dump file lines and write output frame by frame
    f = open(lammps_f, "r")
    o = open(xyz_outname, "w")
    
    line = f.readline()
    while line != '': # EOF in python is empty string
        
        
        if header_token in line: # start of a a header block
            line = f.readline() # time step value
            timestep = line.split()[0]
            for i in range(header_lines-1):
                line = f.readline()
        
        
        # We are at the start of the coordinates block:
        xyz_frame = [] 
        # fill vector relating index in frame to atom id
        indexes_for_ordering = np.zeros(N_atoms, dtype=int)
        counter = 0
        while header_token not in line:
            # Check for EOF
            if len(line.split()) == 0:
                break
            xyz_frame.append(line) # store coordinates of counter-th atom
            indexes_for_ordering[ int(line.split()[atom_id_index_in_line])-1 ] = counter
            counter += 1
            line = f.readline()
            
        
        # write frame
        o.write(str(N_atoms)+"\n")
        o.write("# Timestep: "+str(timestep)+"\n")
        for i in range(len(indexes_for_ordering)):
            coords = xyz_frame[indexes_for_ordering[i]]
            atom_type = idx_to_type[coords.split()[atom_id_index_in_line]]
            vels_string = "  ".join([coords.split()[vx_index_in_line], coords.split()[vy_index_in_line], coords.split()[vz_index_in_line]])
            o.write(atom_type+"   "+vels_string+"\n")
        
    o.close()
    f.close()
    return None

topo_f = "../1-Trajectory/topology.pdb"
vel_f = "../1-Trajectory/velocities.lammpsdump"

idx_to_type = read_topology_pdb(topo_f)
lammps_velocity_dump_to_xyz(vel_f, idx_to_type,"../1-Trajectory/velocities.xyz")

mda_traj = mda.Universe("../1-Trajectory/topology.pdb", "../1-Trajectory/velocities.xyz", dt=0.01) # dt=0.01 ps
mda_traj.dimensions = [21.725, 21.725, 21.725, 90, 90, 90] # PBC box : lx ly lz alpha beta gamma

In [ ]:
def vacf(mda_traj, origin_sep = 100):
    # compute the array at all times (axis = 0), all water molecules (axis = 1), of the normalized dipole vector
    all_vec_O_vel = np.zeros((mda_traj.trajectory.n_frames,np.sum(mda_traj.atoms.names == "O"),3))
    for t in range(mda_traj.trajectory.n_frames):
        all_vec_O_vel[t,:,0] = mda_traj.trajectory[t].positions[mda_traj.atoms.names == "O",0]
        all_vec_O_vel[t,:,1] = mda_traj.trajectory[t].positions[mda_traj.atoms.names == "O",1]
        all_vec_O_vel[t,:,2] = mda_traj.trajectory[t].positions[mda_traj.atoms.names == "O",2]
    
    vv_tcf = np.zeros(mda_traj.trajectory.n_frames)
    vv_tcf_norm = np.zeros(mda_traj.trajectory.n_frames)

    for t0 in range(0, mda_traj.trajectory.n_frames, origin_sep):
        for t in range(t0,mda_traj.trajectory.n_frames):
            # average over all O
            vv_tcf[t-t0] += np.mean(np.sum(all_vec_O_vel[t0,:,:] * all_vec_O_vel[t,:,:], axis=-1))
            vv_tcf_norm[t-t0] += 1
    
    return vv_tcf / vv_tcf_norm

def plot_vacf(vacf, timestep = 0.01):
    """ Plot tcf """
    time = np.asarray([i*timestep for i in range(len(vacf))])
    
    Fig, ax = plt.subplots(1,1, figsize=(12,8))
    ax.plot(time , vacf, c='tab:blue', ls='-', lw=2)
    ax.set_xlabel("Time [ps]", size=18)
    ax.set_ylabel("vacf "+r'[$\AA{}^2$/fs$^2$]', size=18)
    ax.set_xlim([0,2])
    plt.xticks(size=16)
    plt.yticks(size=16)
    plt.grid()
    plt.axhline(y=0, color='k')
    plt.show()
    return None

In [ ]:
vv_tcf = vacf(mda_traj,100)

# Plot the vacf 

integral = np.trapz(vv_tcf,dx=0.01)

# Determine the diffusion coefficient from the vacf integral